In [4]:
import numpy as np
from scipy.optimize import minimize

# Sabit parametreler
m = 1400  # Araç kütlesi (kg)
A = 2.11  # Ön alan (m²)
C_d = 0.337  # Hava direnci katsayısı
r = 0.297  # Tekerlek yarıçapı (m)
C_rr = 0.015  # Yuvarlanma direnci katsayısı
i = 6.6  # Vites oranı
V_batt = 240  # Batarya voltajı (V)
SOC_min, SOC_max = 0.15, 0.95  # SOC aralığı
I_min, I_max = -120, 400  # Akım aralığı (A)
P_cell_max = 10e3  # Yakıt hücresi maksimum gücü (W)
η_rec = 0.75  # Enerji geri kazanım verimliliği
η_BMS = 0.97  # BMS verimliliği
t_race = 3600  # 1 saatlik yarışma süresi (s)
ρ_air = 1.225  # Havanın yoğunluğu (kg/m³)
g = 9.81  # Yerçekimi ivmesi (m/s²)

# Motor parametreleri (örnek olarak iç rotor motoru seçiyoruz)
P_max = 25e3  # Maksimum güç (W)
τ_max = 90  # Maksimum tork (Nm)
n_nominal = 4000  # Nominal devir sayısı (r/min)
η_motor = 0.9  # Motor verimliliği

# Genişletilmiş model fonksiyonları
def calculate_drag_force(C_d, A, ρ_air, velocity):
    return 0.5 * C_d * A * ρ_air * velocity**2

def calculate_rolling_resistance(C_rr, m, g):
    return C_rr * m * g

def calculate_gravitational_force(m, g, slope):
    return m * g * np.sin(np.radians(slope))

def model(I, V, P_cell, C_d, m, slope, T_env, η_rec, η_BMS, velocity):
    F_drag = calculate_drag_force(C_d, A, ρ_air, velocity)  # Hava direnci kuvveti
    F_rr = calculate_rolling_resistance(C_rr, m, g)  # Yuvarlanma direnci kuvveti
    F_gravity = calculate_gravitational_force(m, g, slope)  # Gravitasyon kuvveti (eğim)
   
    F_total = F_drag + F_rr + F_gravity  # Toplam kuvvet
   
    P_motor = V * I * η_motor  # Motor gücü (W)
    τ = τ_max * (P_motor / P_max)  # Tork (Nm)
    n = P_motor / τ  # Devir sayısı (RPM)
   
    E_total = (P_motor / η_rec) * t_race  # Toplam enerji tüketimi (Joule)
    R = (P_cell * t_race) / E_total  # Menzil (m)
   
    return P_motor, τ, n, E_total, R

# Optimizasyon amacı: Toplam enerji tüketimini minimize etmek
def objective(x):
    I, V, velocity = x  # Akım, voltaj ve hız değerleri
    P_motor, τ, n, E_total, R = model(I, V, P_cell_max, C_d, m, 0, 25, η_rec, η_BMS, velocity)
    return E_total / 3.6e6  # Enerji tüketimini kWh cinsine çeviriyoruz

# Kısıtlar
def constraint1(x):
    I, V, velocity = x
    P_motor, _, _, _, _ = model(I, V, P_cell_max, C_d, m, 0, 25, η_rec, η_BMS, velocity)
    return P_max - P_motor  # Motor gücünün maksimumu aşmaması

def constraint2(x):
    I, V, velocity = x
    P_motor, _, _, _, _ = model(I, V, P_cell_max, C_d, m, 0, 25, η_rec, η_BMS, velocity)
    return P_cell_max - V * I  # Yakıt hücresi gücünün maksimumu aşmaması

# Başlangıç tahminleri ve sınırlar
x0 = [I_max / 2, V_batt / 2, 15]  # 15 m/s (54 km/h) başlangıç hızı
bounds = [(50, I_max), (30, V_batt), (10, 30)]  # Voltaj, akım ve hız sınırları

# Kısıtların tanımlanması
constraints = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint2}]

# Optimizasyon işleminin gerçekleştirilmesi
result_refined = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

# Optimizasyon sonucu
result_refined.x, result_refined.fun

#Sonuçların Yorumlanması 
#result_refined.x: Bu dizi, optimizasyon sonucu bulunan en iyi akım, voltaj ve hız değerlerini içerir. 
#result_refined.fun: Bu değer, bu en iyi değerler için elde edilen minimum enerji tüketimini (kWh cinsinden) gösterir.

(array([50., 30., 15.]), np.float64(1.8000000000000007))